# Establishing a database connection

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote

server_name   = "localhost"
database_name = "Everyloop"

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

# Setting up MetaData and declare tables

In [ ]:
from sqlalchemy import MetaData, Table, Column, Integer, String, ForeignKey
metadata_obj = MetaData()

user_table = Table(
    "pythonUsers",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("username", String(30)),
    Column("password", String),
    Column("server_id", ForeignKey('pythonServers.id'), nullable=False)
)

server_table = Table(
    "pythonServers",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("servername", String),
    Column("IP", String(15))
)

for name, table in metadata_obj.tables.items():
    print(f"Table: {name}")

    for column in table.c:
        print(f"{column.name.ljust(15)}{column.type}")

    print()

# Create all metadata tables

In [ ]:
metadata_obj.create_all(engine)

print("Created tables:")

for table in metadata_obj.tables:
    print(table)

# Drop all metadata tables

In [ ]:
metadata_obj.drop_all(engine)

print("Dropped tables:")

for table in metadata_obj.tables:
    print(table)

# Insert server data (multiple rows)

In [ ]:
from sqlalchemy import insert
import random, string

servers = [
    {"servername": "Mario", "IP": "192.168.0.23"},
    {"servername": "Luigi", "IP": "192.168.0.185"},
    {"servername": "Yoshi", "IP": "192.168.0.15"},
    {"servername": "Bowser", "IP": "192.168.0.97"},
]

for server in servers:
    print(f"{server['servername'].ljust(15)}{server['IP']}")

statement = insert(server_table)
print(statement)

with engine.connect() as conn:
    conn.execute(statement, servers)

# Insert user data (single row)

In [ ]:
from sqlalchemy import insert
import random, string

username = input('Enter username:')
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(10))
server_id = random.randint(1, 4)

statement = insert(user_table).values(username=username, password=password, server_id=server_id)

print(statement)
print(statement.compile().params)

with engine.connect() as conn:
    result = conn.execute(statement)
    print(f"primary key value = {result.inserted_primary_key[0]}")

# Select queries
In order to generate select queries we must first import select()

In [ ]:
from sqlalchemy import select

Below are a few example of select queries:

### Select full table data

In [ ]:
query = select(user_table)
print(query)

### Projection

In [ ]:
query = select(user_table.c.username, user_table.c.password, ("Hi " + user_table.c.username + "!").label('greeting'))
print(f"{query}\n\n{query.compile().params}")

### Selection

In [ ]:
query = select(user_table).where(user_table.c.username == 'Fredrik')
print(f"{query}\n\n{query.compile().params}")

### Order by

In [ ]:
query = select(user_table).order_by(user_table.c.username.asc()).limit(3)
print(query)

### Joining tables

In [ ]:
query = select(user_table.c.id, user_table.c.username, server_table.c.servername).join_from(user_table, server_table)
print(query)

### Grouping and aggregation

In [ ]:
from sqlalchemy import func

query = (
    select(server_table.c.servername, server_table.c.IP, func.count(user_table.c.id).label('Number of users'))
    .join_from(user_table, server_table)
    .group_by(server_table.c.servername, server_table.c.IP)
)

print(query)

# Fetch data

In [ ]:
print(f"{query}\n\n{query.compile().params}\n")

with engine.connect() as conn:
    result = conn.execute(query)
    
    for column_name in result.keys():
        print(column_name.upper().ljust(20), end=' ')
    
    print()

    for row in result:
        for field in row:
            print(str(field).ljust(20), end=' ')
        
        print()

# Table reflection

In [ ]:
reflected_table = Table("colors", metadata_obj, autoload_with=engine)

for column in reflected_table.c:
    print(f"{column.name.ljust(20)}{column.type}")


### Query reflected table

In [ ]:
query = select(reflected_table).limit(10)
print(f"{query}\n\n{query.compile().params}\n")